In [2]:
flightData2015 = spark.read.option('inferSchema','true')\
                           .option('header','true')\
                           .csv('data/flight-data/csv/2015-summary.csv')

In [3]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [4]:
flightData2015.sort('count').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#42 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#42 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#52]
      +- FileScan csv [DEST_COUNTRY_NAME#40,ORIGIN_COUNTRY_NAME#41,count#42] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/zach/Projects/databricks-example-notebooks/data/flight-dat..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [5]:
spark.conf.set('spark.sql.shufflepartitions', '5') # Reduce output partitions from 200 to 5

In [6]:
flightData2015.sort('count').take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [7]:
flightData2015.createOrReplaceTempView('flight_data_2015') # Create view from DataFrame

In [8]:
sq = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1) FROM flight_data_2015 GROUP BY DEST_COUNTRY_NAME
""")

In [9]:
df = flightData2015.groupBy('DEST_COUNTRY_NAME').count()

In [10]:
sq.explain() # same plan as dataframe approach

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#40], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#40, 200), ENSURE_REQUIREMENTS, [id=#74]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#40], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#40] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/zach/Projects/databricks-example-notebooks/data/flight-dat..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [15]:
df.explain() # same plan as SQL approach

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#40], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#40, 200), ENSURE_REQUIREMENTS, [id=#87]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#40], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#40] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/zach/Projects/databricks-example-notebooks/data/flight-dat..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




In [12]:
spark.sql("SELECT max(count) FROM flight_data_2015").take(1) # same result as SELECT method

[Row(max(count)=370002)]

In [14]:
from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1) # same result as SQL query

[Row(max(count)=370002)]

In [16]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [18]:
from pyspark.sql.functions import desc
flightData2015.groupBy("DEST_COUNTRY_NAME")\
              .sum("count")\
              .withColumnRenamed("sum(count)", "destination_total")\
              .sort(desc("destination_total"))\
              .limit(5)\
              .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [19]:
flightData2015.groupBy("DEST_COUNTRY_NAME")\
              .sum("count")\
              .withColumnRenamed("sum(count)", "destination_total")\
              .sort(desc("destination_total"))\
              .limit(5)\
              .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#136L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#40,destination_total#136L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#40], functions=[sum(count#42)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#40, 200), ENSURE_REQUIREMENTS, [id=#287]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#40], functions=[partial_sum(count#42)])
            +- FileScan csv [DEST_COUNTRY_NAME#40,count#42] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/zach/Projects/databricks-example-notebooks/data/flight-dat..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


